# Business Understanding
By [THE TIMES OF INDIA](https://timesofindia.indiatimes.com/business/india-business/india-becomes-third-largest-startup-ecosystem-in-the-world/articleshow/85871428.cms), India has emerged as the third largest startup ecosystem in the world after US and China. Following this, our team aims to strategically enter the Indian Startup Ecosystem by leveraging data-driven insights to identify high-potential opportunities. Through comprehensive research and analysis, we seek to gain insight into funding received by startups in India from 2018 to 2021.

### Hypothesis
**Null Hypothesis (Ho):** The sector of a startup has no significant influence on the funding it receives.<br>

**Alternative Hypothesis (Ha):** The sector of a startup has significant influence on the funding it receives.

# Data Understanding
This data provides information into amount of money startups received from 2018 to 2021, the sector of startups, headquaters, what a startup do, the year of establishment, startup name, investors, and stage.<br>

`Feature Description`:
- **Company_Brand:** Name of startup
- **Founded:** Year of establishment
- **HeadQuater:** Location of startup Headquater
- **Sector:** Sector or industry of startup
- **What_it_does:** what the startup does
- **Founders:** Name od founder
- **Investors:** Name of investor
- **Amount:** Amount of investment in USD and INR
- **Statge:** Phase of development (eg. Ideation Stage, Pre-Seed Stage, Seed Stage, Early Stage (Series A, B, etc.))
- **Year:** Year startup received funding

### Analytical Questions
1. How is funding spread across the years?
2. What are the dominant sectors within the Indian startup ecosystem across the years?
3. Are there any emerging sectors that have shown a significant increase in funding year over year?
4. Where in India could be considered the surviving grounds for startups?
5. How does the startup's location influence its funding and growth opportunities?
6. Is there a relationship between what a startup does and the funding it receives?
7. Is there a correlation between the year a startup received funding and the amount of funding it received?
8. Which cities or regions have the highest concentration of funded startups?

### Loading the data

In [1]:
# Import necesary libraries and packages
import pyodbc
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load environment variables from .env file
environment_variables = dotenv_values('.env')
# Access login credentials from  the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("UID")
password = environment_variables.get("PWD")

In [3]:
#  Credection Connection tesr
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# connect to the server with help of pyodbc
connection = pyodbc.connect(connection_string)

In [4]:
# Write querry to retrieve tables from database
query1 = "Select * from dbo.LP1_startup_funding2020"
query2 = "Select * from dbo.LP1_startup_funding2021"

# Retrieve dataset from database with connection created
df_2020 = pd.read_sql(query1, connection)
df_2021 = pd.read_sql(query2, connection)

# Load CSV files
df_2018 = pd.read_csv('Data\startup_funding2018.csv')
df_2019 = pd.read_csv('Data\startup_funding2019.csv')

## Exploring data quality and characteristics

### Data cleaning

In [5]:
# Preview dataframe
df_2020.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,None
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,None
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,NaN,Pre-seed,None
3,NCOME,2020.0,New Delhi,Escrow,Escrow-as-a-service platform,Ritesh Tiwari,"Venture Catalysts, PointOne Capital",400000.0,None,None
4,Gramophone,2016.0,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,"Ashish Rajan Singh, Harshit Gupta, Nishant Mah...","Siana Capital Management, Info Edge",340000.0,None,None


In [6]:
# Add year column to dataframe
df_2020["Year"] = 2020
df_2020.head()


,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10,Year
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,None,2020
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,None,2020
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,NaN,Pre-seed,None,2020
3,NCOME,2020.0,New Delhi,Escrow,Escrow-as-a-service platform,Ritesh Tiwari,"Venture Catalysts, PointOne Capital",400000.0,None,None,2020
4,Gramophone,2016.0,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,"Ashish Rajan Singh, Harshit Gupta, Nishant Mah...","Siana Capital Management, Info Edge",340000.0,None,None,2020


In [7]:
print(df_2020.info(), "\n====================== Null Value Percentage ==========================")
print(df_2020.isna().mean().mul(100), "\n======================= Duplicated rows =========================")
df_2020.loc[df_2020.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Founded        842 non-null    float64
 2   HeadQuarter    961 non-null    object 
 3   Sector         1042 non-null   object 
 4   What_it_does   1055 non-null   object 
 5   Founders       1043 non-null   object 
 6   Investor       1017 non-null   object 
 7   Amount         801 non-null    float64
 8   Stage          591 non-null    object 
 9   column10       2 non-null      object 
 10  Year           1055 non-null   int64  
dtypes: float64(2), int64(1), object(8)
memory usage: 90.8+ KB
None 
====================== Null Value Percentage ==========================
Company_Brand     0.000000
Founded          20.189573
HeadQuarter       8.909953
Sector            1.232227
What_it_does      0.000000
Founders          1.137441
Investor          

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10,Year
145,Krimanshi,2015.0,Jodhpur,Biotechnology company,Krimanshi aims to increase rural income by imp...,Nikhil Bohra,"Rajasthan Venture Capital Fund, AIM Smart City",600000.0,Seed,None,2020
205,Nykaa,2012.0,Mumbai,Cosmetics,Nykaa is an online marketplace for different b...,Falguni Nayar,"Alia Bhatt, Katrina Kaif",NaN,None,None,2020
362,Byju’s,2011.0,Bangalore,EdTech,An Indian educational technology and online tu...,Byju Raveendran,"Owl Ventures, Tiger Global Management",500000000.0,None,None,2020


In the seed stage, startup has a working prototype or minimum viable product (MVP) and is focused on validating the product-market fit. The goal is to attract initial customers and generate early revenue. Key activities carried out in this stage include Product launch, customer acquisition, and iterating based on feedback.
Following this, all duplicated rows with the seed stage are dropped. This is because this is an early stage and expansion is not an option for startups.

In [8]:
# Drop duplicated rows from DataFrame
df_2020.drop_duplicates(keep = "first", inplace = True)

Column10 has 99.8% null values so we drop it

In [9]:
# Drop columns10
df_2020.drop(columns = ["column10"], inplace = True)
# Preview remaining columns 
df_2020.columns

Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage', 'Year'],
      dtype='object')

Amount column has 24% null values which is below a set threshold of 30% so we fill it with the median value

In [10]:
# Fill null values with the median
df_2020.fillna({"Amount": df_2020["Amount"].median()}, inplace = True)
df_2020.isna().sum()

Company_Brand      0
Founded          213
HeadQuarter       94
Sector            13
What_it_does       0
Founders          12
Investor          38
Amount             0
Stage            462
Year               0
dtype: int64

In [11]:
# Preview dataframe
df_2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed


In [12]:
# Add year column to dataframe
df_2021["Year"] = 2021
df_2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A,2021
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None,2021
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D,2021
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C,2021
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed,2021


In [13]:
print(df_2021.info(), "\n====================== Null Value Percentage ======================")
print(df_2021.isna().mean().mul(100), "\n====================== Duplicated rows ======================")
df_2021.loc[df_2021.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1206 non-null   object 
 8   Stage          781 non-null    object 
 9   Year           1209 non-null   int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 94.6+ KB
None 
====================== Null Value Percentage ======================
Company_Brand     0.000000
Founded           0.082713
HeadQuarter       0.082713
Sector            0.000000
What_it_does      0.000000
Founders          0.330852
Investor          5.128205
Amount            0.248139
Stage       

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year
107,Curefoods,2020.0,Bangalore,Food & Beverages,Healthy & nutritious foods and cold pressed ju...,Ankit Nagori,"Iron Pillar, Nordstar, Binny Bansal",$13000000,None,2021
109,Bewakoof,2012.0,Mumbai,Apparel & Fashion,Bewakoof is a lifestyle fashion brand that mak...,Prabhkiran Singh,InvestCorp,$8000000,None,2021
111,FanPlay,2020.0,Computer Games,Computer Games,A real money game app specializing in trivia g...,YC W21,"Pritesh Kumar, Bharat Gupta",Upsparks,$1200000,2021
117,Advantage Club,2014.0,Mumbai,HRTech,Advantage Club is India's largest employee eng...,"Sourabh Deorah, Smiti Bhatt Deorah","Y Combinator, Broom Ventures, Kunal Shah",$1700000,None,2021
119,Ruptok,2020.0,New Delhi,FinTech,Ruptok fintech Pvt. Ltd. is an online gold loa...,Ankur Gupta,Eclear Leasing,$1000000,None,2021
243,Trinkerr,2021.0,Bangalore,Capital Markets,Trinkerr is India's first social trading platf...,"Manvendra Singh, Gaurav Agarwal",Accel India,$6600000,Series A,2021
244,Zorro,2021.0,Gurugram,Social network,Pseudonymous social network platform,"Jasveer Singh, Abhishek Asthana, Deepak Kumar","Vijay Shekhar Sharma, Ritesh Agarwal, Ankiti Bose",$32000000,Seed,2021
245,Ultraviolette,2021.0,Bangalore,Automotive,Create and Inspire the future of sustainable u...,"Subramaniam Narayan, Niraj Rajmohan","TVS Motor, Zoho",$150000000,Series C,2021
246,NephroPlus,2009.0,Hyderabad,Hospital & Health Care,A vision and passion of redefining healthcare ...,Vikram Vuppala,IIFL Asset Management,$24000000,Series E,2021
247,Unremot,2020.0,Bangalore,Information Technology & Services,Unremot is a personal office for consultants!,Shiju Radhakrishnan,Inflection Point Ventures,$700000,Seed,2021


Almost all the duplicated rows in 2021 are startups in their scaling stage. Hence, cannot be dropped. This is because, startups in this stage are focused on growing its customer base by opening branches in different locations to increase market share and revenue.

In [14]:
df_2021["Stage"].unique()

array(['Pre-series A', None, 'Series D', 'Series C', 'Seed', 'Series B',
       'Series E', 'Pre-seed', 'Series A', 'Pre-series B', 'Debt',
       '$1200000', 'Bridge', 'Seed+', 'Series F2', 'Series A+',
       'Series G', 'Series F', 'Series H', 'Series B3', 'PE', 'Series F1',
       'Pre-series A1', '$300000', 'Early seed', 'Series D1', '$6000000',
       '$1000000', 'Seies A', 'Pre-series', 'Series A2', 'Series I'],
      dtype=object)

It could be seen that some of the amount were entered in the stage column due to human errors

In [15]:
# Replace all numeric values with with their corrresponding values in the stage column
df_2021["Amount"].replace("Upsparks", 1200000, inplace = True)
df_2021["Amount"].replace("ah! Ventures", 300000, inplace = True)
df_2021["Amount"].replace("ITO Angel Network, LetsVenture", 300000, inplace = True)
df_2021["Amount"].replace("None", 6000000, inplace = True)
df_2021["Amount"].replace("JITO Angel Network, LetsVenture", 1000000, inplace = True)

In [16]:
df_2021["Amount"].unique()

array(['$1,200,000', '$120,000,000', '$30,000,000', '$51,000,000',
       '$2,000,000', '$188,000,000', '$200,000', 'Undisclosed',
       '$1,000,000', '$3,000,000', '$100,000', '$700,000', '$9,000,000',
       '$40,000,000', '$49,000,000', '$400,000', '$300,000',
       '$25,000,000', '$160,000,000', '$150,000', '$1,800,000',
       '$5,000,000', '$850,000', '$53,000,000', '$500,000', '$1,100,000',
       '$6,000,000', '$800,000', '$10,000,000', '$21,000,000',
       '$7,500,000', '$26,000,000', '$7,400,000', '$1,500,000',
       '$600,000', '$800,000,000', '$17,000,000', '$3,500,000',
       '$15,000,000', '$215,000,000', '$2,500,000', '$350,000,000',
       '$5,500,000', '$83,000,000', '$110,000,000', '$500,000,000',
       '$65,000,000', '$150,000,000,000', '$300,000,000', '$2,200,000',
       '$35,000,000', '$140,000,000', '$4,000,000', '$13,000,000', None,
       '$Undisclosed', '$2000000', '$800000', '$6000000', '$2500000',
       '$9500000', '$13000000', '$5000000', '$8000000',

Amount column consists of non numeric values, and numbers in a string format and needs to be cleaned up

In [17]:
# Clean Amount colum
def clean_amount(Amount):
    try:
        Amount = str(Amount)
        # Remove commas
        Amount = Amount.replace(",", "")
        # Replace USD sign with nothing
        if "$" in Amount:
            Amount = Amount.replace("$", "")
            return round(float(Amount), 2) 
        # If no currency symbol, assume it's already in USD
        else:
            return round(float(Amount), 2)
    except ValueError:
        # For non-numeric entries, return NaN
        return np.nan

# Apply the clean_amount function to the 'amount' column
df_2021["Amount"] = df_2021["Amount"].apply(clean_amount)

In [18]:
df_2021.fillna({"Amount": df_2021["Amount"].median()}, inplace = True)
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1209 non-null   float64
 8   Stage          781 non-null    object 
 9   Year           1209 non-null   int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 94.6+ KB


In [19]:
# Preview dataframe
df_2018.head()

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...


In [20]:
# Add year column to dataframe
df_2018["Year"] = 2018
df_2018.head()

,Company Name,Industry,Round/Series,Amount,Location,About Company,Year
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f...",2018
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...,2018
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India,2018
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...,2018
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...,2018


In [21]:
print(df_2018.info(), "\n====================== Null Value Percentage ======================")
print(df_2018.isna().mean().mul(100), "\n====================== Duplicated rows ======================")
df_2018.loc[df_2018.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company Name   526 non-null    object
 1   Industry       526 non-null    object
 2   Round/Series   526 non-null    object
 3   Amount         526 non-null    object
 4   Location       526 non-null    object
 5   About Company  526 non-null    object
 6   Year           526 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 28.9+ KB
None 
====================== Null Value Percentage ======================
Company Name     0.0
Industry         0.0
Round/Series     0.0
Amount           0.0
Location         0.0
About Company    0.0
Year             0.0
dtype: float64 
====================== Duplicated rows ======================


,Company Name,Industry,Round/Series,Amount,Location,About Company,Year
348,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f...",2018


Startup is in seed stage so we drop it

In [22]:
# Drop duplicated rows from DataFrame
df_2018.drop_duplicates(keep = "first", inplace = True)

Rename columns to match that of the other DataFrames to be merged

In [23]:
# Rename columns
df_2018.rename(columns={'Company Name':'Company_Brand','Industry':'Sector','Round/Series':'Stage','Location':'HeadQuarter','About Company':'What_it_does'} ,inplace=True)

In [24]:
df_2018["Amount"].unique()

array(['250000', '₹40,000,000', '₹65,000,000', '2000000', '—', '1600000',
       '₹16,000,000', '₹50,000,000', '₹100,000,000', '150000', '1100000',
       '₹500,000', '6000000', '650000', '₹35,000,000', '₹64,000,000',
       '₹20,000,000', '1000000', '5000000', '4000000', '₹30,000,000',
       '2800000', '1700000', '1300000', '₹5,000,000', '₹12,500,000',
       '₹15,000,000', '500000', '₹104,000,000', '₹45,000,000', '13400000',
       '₹25,000,000', '₹26,400,000', '₹8,000,000', '₹60,000', '9000000',
       '100000', '20000', '120000', '₹34,000,000', '₹342,000,000',
       '$143,145', '₹600,000,000', '$742,000,000', '₹1,000,000,000',
       '₹2,000,000,000', '$3,980,000', '$10,000', '₹100,000',
       '₹250,000,000', '$1,000,000,000', '$7,000,000', '$35,000,000',
       '₹550,000,000', '$28,500,000', '$2,000,000', '₹240,000,000',
       '₹120,000,000', '$2,400,000', '$30,000,000', '₹2,500,000,000',
       '$23,000,000', '$150,000', '$11,000,000', '₹44,000,000',
       '$3,240,000', '₹60

Amount column consist of both INR and USD. Hence, need to cleaned 

In [25]:
# Clean Amount colum and convert Indian Rupee to USD currency
def clean_amount(Amount):
    try:
        Amount = str(Amount)
        # Remove commas
        Amount = Amount.replace(",", "")
        # Check if the amount is in INR and convert to USD assuming 1 USD = 70 INR
        if "₹" in Amount:
            Amount = Amount.replace("₹", "")
            return round(float(Amount) / 70, 2)
        # Check if the amount is in USD
        elif "$" in Amount:
            Amount = Amount.replace("$", "")
            return round(float(Amount), 2) 
        # If no currency symbol, assume it's already in USD
        else:
            return round(float(Amount), 2)
    except ValueError:
        # For non-numeric entries, return NaN
        return np.nan

# Apply the clean_amount function to the 'amount' column
df_2018["Amount"] = df_2018["Amount"].apply(clean_amount)

In [26]:
df_2018.fillna({"Amount": df_2018["Amount"].median()}, inplace = True)
df_2018.info()


<class 'pandas.core.frame.DataFrame'>
Index: 525 entries, 0 to 525
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  525 non-null    object 
 1   Sector         525 non-null    object 
 2   Stage          525 non-null    object 
 3   Amount         525 non-null    float64
 4   HeadQuarter    525 non-null    object 
 5   What_it_does   525 non-null    object 
 6   Year           525 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 32.8+ KB


In [27]:
# Preview dataframe
df_2019.head()

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN
1,Ruangguru,2014.0,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C
2,Eduisfun,NaN,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding
3,HomeLane,2014.0,Chennai,Interior design,Provides interior designing solutions,"Srikanth Iyer, Rama Harinath","Evolvence India Fund (EIF), Pidilite Group, FJ...","$30,000,000",Series D
4,Nu Genes,2004.0,Telangana,AgriTech,"It is a seed company engaged in production, pr...",Narayana Reddy Punyala,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN


In [28]:
# Add year column to dataframe
df_2019["Year"] = 2019
df_2019.head()

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage,Year
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN,2019
1,Ruangguru,2014.0,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C,2019
2,Eduisfun,NaN,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding,2019
3,HomeLane,2014.0,Chennai,Interior design,Provides interior designing solutions,"Srikanth Iyer, Rama Harinath","Evolvence India Fund (EIF), Pidilite Group, FJ...","$30,000,000",Series D,2019
4,Nu Genes,2004.0,Telangana,AgriTech,"It is a seed company engaged in production, pr...",Narayana Reddy Punyala,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN,2019


In [29]:
print(df_2019.info(), "\n====================== Null Value Percentage ======================")
print(df_2019.isna().mean().mul(100), "\n====================== Duplicated rows ======================")
df_2019.loc[df_2019.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company/Brand  89 non-null     object 
 1   Founded        60 non-null     float64
 2   HeadQuarter    70 non-null     object 
 3   Sector         84 non-null     object 
 4   What it does   89 non-null     object 
 5   Founders       86 non-null     object 
 6   Investor       89 non-null     object 
 7   Amount($)      89 non-null     object 
 8   Stage          43 non-null     object 
 9   Year           89 non-null     int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 7.1+ KB
None 
====================== Null Value Percentage ======================
Company/Brand     0.000000
Founded          32.584270
HeadQuarter      21.348315
Sector            5.617978
What it does      0.000000
Founders          3.370787
Investor          0.000000
Amount($)         0.000000
Stage            

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage,Year


In [30]:
df_2019.rename(columns={'Company/Brand':'Company_Brand','Amount($)':'Amount', 'What it does': 'What_it_does'}, inplace=True)

In [31]:
df_2019["Amount"].unique()

array(['$6,300,000', '$150,000,000', '$28,000,000', '$30,000,000',
       '$6,000,000', 'Undisclosed', '$1,000,000', '$20,000,000',
       '$275,000,000', '$22,000,000', '$5,000,000', '$140,500',
       '$540,000,000', '$15,000,000', '$182,700', '$12,000,000',
       '$11,000,000', '$15,500,000', '$1,500,000', '$5,500,000',
       '$2,500,000', '$140,000', '$230,000,000', '$49,400,000',
       '$32,000,000', '$26,000,000', '$150,000', '$400,000', '$2,000,000',
       '$100,000,000', '$8,000,000', '$100,000', '$50,000,000',
       '$120,000,000', '$4,000,000', '$6,800,000', '$36,000,000',
       '$5,700,000', '$25,000,000', '$600,000', '$70,000,000',
       '$60,000,000', '$220,000', '$2,800,000', '$2,100,000',
       '$7,000,000', '$311,000,000', '$4,800,000', '$693,000,000',
       '$33,000,000'], dtype=object)

Amount column consists of non numeric values, and numbers in a string format which needs to be cleaned up

In [32]:
# Clean Amount colum
def clean_amount(Amount):
    try:
        Amount = str(Amount)
        # Remove commas
        Amount = Amount.replace(",", "")
        # Replace USD sign with nothing
        if "$" in Amount:
            Amount = Amount.replace("$", "")
            return round(float(Amount), 2) 
        # If no currency symbol, assume it's already in USD
        else:
            return round(float(Amount), 2)
    except ValueError:
        # For non-numeric entries, return NaN
        return np.nan

# Apply the clean_amount function to the 'amount' column
df_2019["Amount"] = df_2019["Amount"].apply(clean_amount)

In [33]:
df_2019.fillna({"Amount": df_2019["Amount"].median()}, inplace = True)
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  89 non-null     object 
 1   Founded        60 non-null     float64
 2   HeadQuarter    70 non-null     object 
 3   Sector         84 non-null     object 
 4   What_it_does   89 non-null     object 
 5   Founders       86 non-null     object 
 6   Investor       89 non-null     object 
 7   Amount         89 non-null     float64
 8   Stage          43 non-null     object 
 9   Year           89 non-null     int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 7.1+ KB


In [34]:
print(df_2020.columns)
print(df_2021.columns)
print(df_2018.columns)
print(df_2019.columns)

Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage', 'Year'],
      dtype='object')
Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage', 'Year'],
      dtype='object')
Index(['Company_Brand', 'Sector', 'Stage', 'Amount', 'HeadQuarter',
       'What_it_does', 'Year'],
      dtype='object')
Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage', 'Year'],
      dtype='object')


Columns have same names and can be concatenated

In [35]:
# Combine 2020 and 2021 dataframes into a single dataframe
df_combined = pd.concat([df_2020, df_2021, df_2018, df_2019], axis = 0)
# Preview combined dataframe
df_combined.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,2020
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,2020
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,3000000.0,Pre-seed,2020
3,NCOME,2020.0,New Delhi,Escrow,Escrow-as-a-service platform,Ritesh Tiwari,"Venture Catalysts, PointOne Capital",400000.0,None,2020
4,Gramophone,2016.0,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,"Ashish Rajan Singh, Harshit Gupta, Nishant Mah...","Siana Capital Management, Info Edge",340000.0,None,2020


In [36]:
print(df_combined.info(), "\n====================== Null Value Percentage ======================")
print(df_combined.isna().mean().mul(100), "\n====================== Duplicated rows ======================")
df_combined.loc[df_combined.duplicated()]

<class 'pandas.core.frame.DataFrame'>
Index: 2875 entries, 0 to 88
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  2875 non-null   object 
 1   Founded        2107 non-null   float64
 2   HeadQuarter    2761 non-null   object 
 3   Sector         2857 non-null   object 
 4   What_it_does   2875 non-null   object 
 5   Founders       2331 non-null   object 
 6   Investor       2250 non-null   object 
 7   Amount         2875 non-null   float64
 8   Stage          1939 non-null   object 
 9   Year           2875 non-null   int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 247.1+ KB
None 
====================== Null Value Percentage ======================
Company_Brand     0.000000
Founded          26.713043
HeadQuarter       3.965217
Sector            0.626087
What_it_does      0.000000
Founders         18.921739
Investor         21.739130
Amount            0.000000
Stage            3

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year
107,Curefoods,2020.0,Bangalore,Food & Beverages,Healthy & nutritious foods and cold pressed ju...,Ankit Nagori,"Iron Pillar, Nordstar, Binny Bansal",13000000.0,None,2021
109,Bewakoof,2012.0,Mumbai,Apparel & Fashion,Bewakoof is a lifestyle fashion brand that mak...,Prabhkiran Singh,InvestCorp,8000000.0,None,2021
111,FanPlay,2020.0,Computer Games,Computer Games,A real money game app specializing in trivia g...,YC W21,"Pritesh Kumar, Bharat Gupta",1200000.0,$1200000,2021
117,Advantage Club,2014.0,Mumbai,HRTech,Advantage Club is India's largest employee eng...,"Sourabh Deorah, Smiti Bhatt Deorah","Y Combinator, Broom Ventures, Kunal Shah",1700000.0,None,2021
119,Ruptok,2020.0,New Delhi,FinTech,Ruptok fintech Pvt. Ltd. is an online gold loa...,Ankur Gupta,Eclear Leasing,1000000.0,None,2021
243,Trinkerr,2021.0,Bangalore,Capital Markets,Trinkerr is India's first social trading platf...,"Manvendra Singh, Gaurav Agarwal",Accel India,6600000.0,Series A,2021
244,Zorro,2021.0,Gurugram,Social network,Pseudonymous social network platform,"Jasveer Singh, Abhishek Asthana, Deepak Kumar","Vijay Shekhar Sharma, Ritesh Agarwal, Ankiti Bose",32000000.0,Seed,2021
245,Ultraviolette,2021.0,Bangalore,Automotive,Create and Inspire the future of sustainable u...,"Subramaniam Narayan, Niraj Rajmohan","TVS Motor, Zoho",150000000.0,Series C,2021
246,NephroPlus,2009.0,Hyderabad,Hospital & Health Care,A vision and passion of redefining healthcare ...,Vikram Vuppala,IIFL Asset Management,24000000.0,Series E,2021
247,Unremot,2020.0,Bangalore,Information Technology & Services,Unremot is a personal office for consultants!,Shiju Radhakrishnan,Inflection Point Ventures,700000.0,Seed,2021


In [37]:
df_combined["HeadQuarter"].unique()

array(['Chennai', 'Bangalore', 'Pune', 'New Delhi', 'Indore', 'Hyderabad',
       'Gurgaon', 'Belgaum', 'Noida', 'Mumbai', 'Andheri', 'Jaipur',
       'Ahmedabad', 'Kolkata', 'Tirunelveli, Tamilnadu', 'Thane', None,
       'Singapore', 'Gurugram', 'Gujarat', 'Haryana', 'Kerala', 'Jodhpur',
       'Jaipur, Rajastan', 'Delhi', 'Frisco, Texas, United States',
       'California', 'Dhingsara, Haryana', 'New York, United States',
       'Patna', 'San Francisco, California, United States',
       'San Francisco, United States', 'San Ramon, California',
       'Paris, Ile-de-France, France', 'Plano, Texas, United States',
       'Sydney', 'San Francisco Bay Area, Silicon Valley, West Coast',
       'Bangaldesh', 'London, England, United Kingdom',
       'Sydney, New South Wales, Australia', 'Milano, Lombardia, Italy',
       'Palmwoods, Queensland, Australia', 'France',
       'San Francisco Bay Area, West Coast, Western US',
       'Trivandrum, Kerala, India', 'Cochin', 'Samastipur, Bihar',


HeadQuarter column needs to be cleaned up to get just the headquarters' locations as some of its rows included country and town name

In [38]:
df_combined["HeadQuarter"] = df_combined["HeadQuarter"].str.split(",", expand = True)[0]
df_combined["HeadQuarter"] = df_combined["HeadQuarter"].str.split("\t", expand = True)[0]

df_combined["HeadQuarter"].unique()

array(['Chennai', 'Bangalore', 'Pune', 'New Delhi', 'Indore', 'Hyderabad',
       'Gurgaon', 'Belgaum', 'Noida', 'Mumbai', 'Andheri', 'Jaipur',
       'Ahmedabad', 'Kolkata', 'Tirunelveli', 'Thane', None, 'Singapore',
       'Gurugram', 'Gujarat', 'Haryana', 'Kerala', 'Jodhpur', 'Delhi',
       'Frisco', 'California', 'Dhingsara', 'New York', 'Patna',
       'San Francisco', 'San Ramon', 'Paris', 'Plano', 'Sydney',
       'San Francisco Bay Area', 'Bangaldesh', 'London', 'Milano',
       'Palmwoods', 'France', 'Trivandrum', 'Cochin', 'Samastipur',
       'Irvine', 'Tumkur', 'Newcastle Upon Tyne', 'Shanghai', 'Jiaxing',
       'Rajastan', 'Kochi', 'Ludhiana', 'Dehradun', 'San Franciscao',
       'Tangerang', 'Berlin', 'Seattle', 'Riyadh', 'Seoul', 'Bangkok',
       'Kanpur', 'Chandigarh', 'Warangal', 'Hyderebad', 'Odisha', 'Bihar',
       'Goa', 'Tamil Nadu', 'Uttar Pradesh', 'Bhopal', 'Banglore',
       'Coimbatore', 'Bengaluru', 'Ahmadabad', 'Small Towns', 'Rajsamand',
       'Ranchi'

In [39]:
df_combined.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,2020
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,2020
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,3000000.0,Pre-seed,2020
3,NCOME,2020.0,New Delhi,Escrow,Escrow-as-a-service platform,Ritesh Tiwari,"Venture Catalysts, PointOne Capital",400000.0,None,2020
4,Gramophone,2016.0,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,"Ashish Rajan Singh, Harshit Gupta, Nishant Mah...","Siana Capital Management, Info Edge",340000.0,None,2020


In [40]:
df_combined.isna().mean().mul(100)

Company_Brand     0.000000
Founded          26.713043
HeadQuarter       3.965217
Sector            0.626087
What_it_does      0.000000
Founders         18.921739
Investor         21.739130
Amount            0.000000
Stage            32.556522
Year              0.000000
dtype: float64

Drop columns that do not contribute to answering the research questions or achieving the analysis goals

In [41]:
df_combined.drop(columns = ["Founded", "Founders", "Stage", "What_it_does"], inplace = True)
df_combined.head()

,Company_Brand,HeadQuarter,Sector,Investor,Amount,Year
0,Aqgromalin,Chennai,AgriTech,Angel investors,200000.0,2020
1,Krayonnz,Bangalore,EdTech,GSF Accelerator,100000.0,2020
2,PadCare Labs,Pune,Hygiene management,Venture Center,3000000.0,2020
3,NCOME,New Delhi,Escrow,"Venture Catalysts, PointOne Capital",400000.0,2020
4,Gramophone,Indore,AgriTech,"Siana Capital Management, Info Edge",340000.0,2020


We drop null values in HeadQuarter and sector columns. This is because dropping them won't have anay significant effect on the analysis

In [42]:
df_combined.dropna(subset = ["HeadQuarter", "Sector"], inplace = True)

Fill investor column with the mose frquent value

In [43]:
# Fill the 'Sector' column with the most frequent value
df_combined.fillna({"Investor": df_combined["Investor"].mode()[0]} , inplace = True)
print(df_combined.info(), "\n====================== Null Value Percentage ======================")
print(df_combined.isna().mean().mul(100))
df_combined.head()


<class 'pandas.core.frame.DataFrame'>
Index: 2746 entries, 0 to 88
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  2746 non-null   object 
 1   HeadQuarter    2746 non-null   object 
 2   Sector         2746 non-null   object 
 3   Investor       2746 non-null   object 
 4   Amount         2746 non-null   float64
 5   Year           2746 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 150.2+ KB
None 
====================== Null Value Percentage ======================
Company_Brand    0.0
HeadQuarter      0.0
Sector           0.0
Investor         0.0
Amount           0.0
Year             0.0
dtype: float64


,Company_Brand,HeadQuarter,Sector,Investor,Amount,Year
0,Aqgromalin,Chennai,AgriTech,Angel investors,200000.0,2020
1,Krayonnz,Bangalore,EdTech,GSF Accelerator,100000.0,2020
2,PadCare Labs,Pune,Hygiene management,Venture Center,3000000.0,2020
3,NCOME,New Delhi,Escrow,"Venture Catalysts, PointOne Capital",400000.0,2020
4,Gramophone,Indore,AgriTech,"Siana Capital Management, Info Edge",340000.0,2020


## Hypothesis Testing